<a href="https://colab.research.google.com/github/ydaigo/BERT_HANDSON/blob/master/BERT_HANDS_ON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERTのハンズオン
[transformers](https://github.com/huggingface/transformers)を動かします。

In [0]:
#準備
!pip install transformers
!pip install mecab-python3
from transformers import pipeline,AutoTokenizer,BertTokenizer,AutoModelForSequenceClassification,BertJapaneseTokenizer, BertForMaskedLM

## トークンナイザー
トークンナイザーは文章を単語ごとに分割し数字を割り当てる。

In [0]:
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
sequence = "私はエンジニアである。"
tokenized_sequence = tokenizer.tokenize(sequence)
print(tokenized_sequence)
encoded_sequence = tokenizer.encode(sequence)
print(encoded_sequence)

#感情分類
感情判定のモデルです。  
0:ポジティブ
1:ネガティブ

[bert-base-japanese-char-whole-word-masking](https://huggingface.co/bert-base-japanese-char-whole-word-masking)というモデルを自作データセットでファインチューニングをしました。  
[ファインチューニングしたモデル](daigo/bert-base-japanese-sentiment)

[このサイト](https://qiita.com/kenta1984/items/7f3a5d859a15b20657f3)を参考にしました。

In [0]:
model = AutoModelForSequenceClassification.from_pretrained('daigo/bert-base-japanese-sentiment') 
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
nlp = pipeline("sentiment-analysis",model=model,tokenizer=tokenizer)
print(nlp("私は幸せである。"))

# バンダイナムコが公開したモデルを使ってみる。
[リポジトリ](https://github.com/BandaiNamcoResearchInc/DistilBERT-base-jp?utm_campaign=Weekly%20Kaggle%20News&utm_medium=email&utm_source=Revue%20newsletter)

In [0]:
nlp = pipeline("fill-mask",model="bandainamco-mirai/distilbert-base-japanese",tokenizer="cl-tohoku/bert-base-japanese-whole-word-masking")
nlp(f"私は北陸のシステムエンジニアだ。私は機械学習を{nlp.tokenizer.mask_token}している。")

# いろいろな調整済みの学習済みモデルを使ってみよう。
[参考にしたサイト](https://huggingface.co/transformers/usage.html)

###文章要約
モデル  
bart-large-cnn


文章はアナと雪の女王の要約  
https://www.imdb.com/title/tt2294629/plotsummary

In [0]:
from transformers import pipeline
text = """When the newly crowned Queen Elsa accidentally uses her power to turn things into ice to curse her home in infinite winter,
           her sister Anna teams up with a mountain man, his playful reindeer, and a snowman to change the weather condition."""
summarizer = pipeline("summarization",model="bart-large-cnn")
summarizer(text, min_length=5, max_length=30)

###質問解答
モデル  
[bert-large-uncased-whole-word-masking-finetuned-squad](https://huggingface.co/bert-large-uncased-whole-word-masking-finetuned-squad)  
トークンナイザー  
[bert-base-uncased](https://huggingface.co/bert-base-uncased)

In [0]:
from transformers import pipeline

nlp = pipeline("question-answering",model="distilbert-base-cased-distilled-squad")

context = text
print(nlp(question="Who is Queen?", context=context))

###文章生成  
モデル
[gpt2](https://huggingface.co/gpt2#)

In [0]:
from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelWithLMHead.from_pretrained("gpt2")

sequence = f"I am System Engineer. But"

input = tokenizer.encode(sequence, return_tensors="pt")
generated = model.generate(input, max_length=500, do_sample=True)

resulting_string = tokenizer.decode(generated.tolist()[0])
print(resulting_string)